In [ ]:
!pip install pytesseract evaluate tqdm transformers datasets rouge-score accelerate nltk tensorboard jupyter-black py7zr --upgrade
!apt-get install git --yes
!apt-get install git-lfs --yes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.8 MB/s eta 0:00:

In [ ]:
import json
import pandas
import jupyter_black
from datetime import timedelta
from datasets import Dataset

In [ ]:
with open("data.json", "r") as f:
    data = json.load(f)

# create a dataframe
df = pandas.DataFrame(data)
df

,id,task,question,maximum_score,scoring_guide,answer,score
0,1,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...",malloc() is a C function used for dynamic memo...,6
1,2,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...",malloc() is a C function used for dynamic memo...,4
2,3,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...","After calling malloc(), free() must be also ca...",2
3,4,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...",The memory allocated with new must be dealloca...,0
4,5,Rate the answer to the question.,What is the purpose of the 'const' keyword in ...,3,"[{'point': 2, 'criteria': 'The 'const' keyword...",The 'const' keyword in C++ is used to indicate...,2
5,6,Rate the answer to the question.,What is the purpose of the 'const' keyword in ...,3,"[{'point': 2, 'criteria': 'The 'const' keyword...",The 'const' keyword in C++ is used to indicate...,2


In [ ]:
data = Dataset.from_pandas(df).train_test_split(test_size=0.1)

In [ ]:
print(data['train'].num_rows)

5


Ezidáig az adatok lettek egy datasetbe feldolgozva, innentől a modell felkészítése jön a trainingre

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
MODEL_NAME = "C++ teacher AI"  # the name of your model
MODEL_ID = "google/flan-t5-large"  # the id of the base model we will train (can be small, base, large, xl, etc.) (the bigger - the more GPU memory you need)
NUM_TRAIN_EPOCHS = 4  # number of epochs to train

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [ ]:
merged_prompts = ""
line_break = '\n'


for i in range(4):
  print("i: " + str(i) + "max_score" + str(data[i]['maximum_score']))
  task = data[i]['task']
  question = data[i]['question']
  max_score = data[i]['maximum_score']
  scoring_guide = data[i]['scoring_guide']
  answer = data[i]['answer']
  score = data[i]['score']
  line_break = '\n'

  prompt = f"""
  {task}

  Question:
  {question}

  Maximum score:
  {max_score}

  Scoring guide:
  {line_break.join(str(str(z['point']) + " points - " + z['criteria']) for z in scoring_guide)}

  Answer:
  {answer}

  Score:
  {score if i < 3 else ""}
  """
  merged_prompts = merged_prompts + f'\n\n\n{prompt}'

print(merged_prompts)


i: 0max_score6
i: 1max_score6
i: 2max_score6
i: 3max_score6




  Rate the answer to the question.

  Question:
  Explain the difference between malloc() and new in C++.

  Maximum score:
  6

  Scoring guide:
  2 points - malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically.
2 points - Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.
2 points - new is type-safe, whereas malloc() returns a void pointer, requiring explicit type casting.

  Answer:
  malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically. new not only allocates memory but also calls the constructor for an object. Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. Additionally, new is type-sa

In [ ]:
ind = 0

task = data['test'][ind]['task']
question = data['test'][ind]['question']
max_score = data['test'][ind]['maximum_score']
scoring_guide = data['test'][ind]['scoring_guide']
answer = data['test'][ind]['answer']
score = data['test'][ind]['score']
line_break = '\n'

prompt = f"""
{task}

Question:
{question}

Maximum score:
{max_score}

Scoring guide:
{line_break.join(str(str(i['point']) + " points - " + i['criteria']) for i in scoring_guide)}

Answer:
{answer}

Score:


"""

In [ ]:
inputs = tokenizer(merged_prompts, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

print(f'MODEL_GENERATION - ZERO SHOT:\n{output}')

MODEL_GENERATION - ZERO SHOT:
2


In [ ]:
def tokenize_function (example):
  print(len(example['question']))
  print(type(example))
  line_break = '\n'
  size = len(example['id'])

  prompt = [
      f"""
        {example['task'][i]}

        Question:
        {example['question'][i]}

        Maximum score:
        {example['maximum_score'][i]}

        Scoring guide:
        {line_break.join(str(str(z['point']) + ' points - ' + z['criteria']) for z in example['scoring_guide'][i])}

        Answer:
        {example['answer'][i]}

        Score:

      """
      for i in range(size)]

  for i in range(size):
    example['score'][i] = str(example['score'][i])

  example['input_ids'] = tokenizer (prompt, padding="max_length", truncation =True, return_tensors="pt").input_ids
  example['labels'] = tokenizer (example['score'], padding="max_length", truncation=True, return_tensors="pt").input_ids

  return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = data.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'])

print(tokenized_datasets)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

5
<class 'datasets.formatting.formatting.LazyBatch'>


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1
<class 'datasets.formatting.formatting.LazyBatch'>
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1
    })
})


In [ ]:
from pprint import pprint

#Check the shapes

print("Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

pprint(tokenized_datasets['train'][1])

Shapes of the datasets:
Training: (5, 2)
Test: (1, 2)
{'input_ids': [13002,
               8,
               1525,
               12,
               8,
               822,
               5,
               11860,
               10,
               25488,
               8,
               1750,
               344,
               1460,
               5133,
               9960,
               11,
               126,
               16,
               205,
               16702,
               5,
               24210,
               2604,
               10,
               431,
               10711,
               1007,
               1539,
               10,
               204,
               979,
               3,
               18,
               1460,
               5133,
               9960,
               19,
               3,
               9,
               205,
               1681,
               261,
               21,
               4896,
               2594,
               17127,
   

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
output_dir = f'./first-ever-training'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1,

)

trainer = Trainer(
    model=model,
    args=training_args,

    train_dataset=tokenized_datasets['train']
)

In [ ]:
trainer.train()

OutOfMemoryError: ignored